# Implementación de una Red Neuronal utilizando Pytorch para Fashion-MNIST

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch import optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cargar los datos
train_data = pd.read_csv('/content/drive/MyDrive/datasets/fashion-mnist_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/datasets/fashion-mnist_test.csv')

##### Reshape de las Imágenes
Cuando cargamos imágenes desde un CSV, cada imagen está representada como una fila de números en el CSV, que son los valores de los píxeles. Para un dataset como Fashion-MNIST, cada imagen tiene un tamaño de 28x28 píxeles. Sin embargo, al cargarlas desde el CSV, cada imagen se convierte en un array de 784 elementos (28 multiplicado por 28). Por lo tanto, necesitamos reshape estos arrays para transformarlos de nuevo en la estructura de imagen 2D que representa su formato original.

Esto es importante porque las operaciones que realizamos en imágenes (como visualización y convoluciones en redes neuronales) esperan datos en forma de matriz o en un formato de varias dimensiones (altura, ancho, canales). Al hacer reshape(-1, 28, 28, 1), estamos configurando los datos para tener la siguiente forma:

-1: Infiera la dimensión de este eje basándose en la longitud de los datos y las otras dimensiones especificadas. En práctica, esto se traduce en el número de imágenes.
28, 28: Estas son las dimensiones de cada imagen (altura x ancho).
1: Esto indica el número de canales de color; en este caso, 1, porque las imágenes son en escala de grises.

In [ ]:
# Separar etiquetas y imágenes
train_labels = train_data['label'].values
train_images = train_data.drop('label', axis=1).values.reshape(-1, 28, 28, 1) / 255.0
test_labels = test_data['label'].values
test_images = test_data.drop('label', axis=1).values.reshape(-1, 28, 28, 1) / 255.0

In [ ]:
class CustomFashionMNISTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


##### Transformaciones
Las transformaciones se utilizan para realizar operaciones de preprocesamiento y aumento de datos en las imágenes. Esto puede incluir normalización, recorte, rotación, y otras formas de alteración de la imagen que ayudan a generalizar y mejorar el entrenamiento de modelos de machine learning.

En el código que hemos definido, usamos una serie de transformaciones:

ToPILImage(): Convierte un array de numpy o un tensor de PyTorch en una imagen PIL. Esto es necesario porque muchas de las transformaciones en torchvision esperan imágenes en formato PIL.
ToTensor(): Convierte la imagen PIL (o numpy.ndarray) en un tensor de PyTorch. Esto también cambia el rango de los píxeles de 0-255 a 0-1, reescalandolos mediante una división por 255.
Normalize((0.5,), (0.5,)): Normaliza un tensor de imagen con una media y una desviación estándar. En este caso, la normalización (0.5, 0.5) transforma los píxeles a un rango de [-1, 1], facilitando la convergencia durante el entrenamiento al centrar los datos.
Estas transformaciones son cruciales para preparar los datos de manera que sean más efectivos para entrenar redes neuronales, ayudando a que el modelo aprenda más eficientemente.

In [ ]:
# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convertir numpy arrays a PILImage para transformar
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Crear datasets
train_dataset = CustomFashionMNISTDataset(train_images, train_labels, transform=transform)
test_dataset = CustomFashionMNISTDataset(test_images, test_labels, transform=transform)

# Creando los data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


##### Definición de la red neuronal

In [ ]:
class FashionMNISTModel(nn.Module):
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        self.fc1 = nn.Linear(784, 256)  # Capa de entrada (28*28 = 784)
        self.fc2 = nn.Linear(256, 128)  # Primera capa oculta
        self.fc3 = nn.Linear(128, 64)   # Segunda capa oculta
        self.fc4 = nn.Linear(64, 10)    # Capa de salida (10 clases)

    def forward(self, x):
        x = x.view(x.shape[0], -1)  # Aplanar la imagen
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

2. Instanciar el modelo, el criterio de pérdida y el optimizador


In [ ]:
model = FashionMNISTModel()
criterion = nn.CrossEntropyLoss()  # Usado comúnmente para clasificación multiclase
optimizer = optim.Adam(model.parameters(), lr=0.003)  # Optimizador Adam


3. Entrenar la red


In [ ]:
epochs = 10  # Número de épocas para entrenar
for epoch in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()           # Limpia los gradientes
        output = model(images)          # Pasa las imágenes por la red
        loss = criterion(output, labels)  # Calcula la pérdida
        loss.backward()                 # Propaga la pérdida hacia atrás
        optimizer.step()                # Actualiza los parámetros
        running_loss += loss.item()
    print(f"Epoch {epoch+1} - Training loss: {running_loss/len(train_loader)}")


Epoch 1 - Training loss: 0.5233497702395484
Epoch 2 - Training loss: 0.39506281205395394
Epoch 3 - Training loss: 0.3645390493751589
Epoch 4 - Training loss: 0.3363837057958915
Epoch 5 - Training loss: 0.3187387602955802
Epoch 6 - Training loss: 0.30826748512002194
Epoch 7 - Training loss: 0.2951043083358294
Epoch 8 - Training loss: 0.2870108994053625
Epoch 9 - Training loss: 0.27647093009910606
Epoch 10 - Training loss: 0.2688553388685242


4. Evaluación del modelo

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy}%')

Accuracy on test set: 88.82%
